In [50]:
from bs4 import BeautifulSoup
import json
import requests
import random

import pandas as pd
import math, time
import re
import numpy as np

# pd.set_option("display.max_colwidth",None)

# TripAdvisor


In [51]:
# with open("user-agents_1.txt", "r+") as file:
#     user_agents = file.readlines()
#     user_agents = [user_agent.replace("\n","").strip() for user_agent in user_agents]

In [69]:
# JSON dosyasını oku
with open("user-agents.json", "r") as f:
    user_agents_data = json.load(f)

# User-Agent listesini oluştur
user_agents = [entry["ua"] for entry in user_agents_data]

In [70]:
with open("valid-proxy-list.txt","r") as f:
    prxs = f.read().split("\n")

proxies =[]

for prx in prxs:
    if prx not in proxies:
        proxies.append(prx)

In [71]:
def fetch_html(url):

    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/",
    }

    try:
        proxy = random.choice(proxies)
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # HTTP hatalarını kontrol et
        return response.content
    except requests.exceptions.HTTPError as err:
        print(f"HTTP hatası: {err}")
        return None


def parse_html(html_content):
    return BeautifulSoup(html_content, 'lxml')


def get_namesandcount(soup):
    hotel_names_with_numbers = [hotel.get_text(strip=True) for hotel in soup.find_all(name="div", class_="nBrpc o W")]

    cleaned_hotel_names = [re.sub(r'[^\w\s]', '', re.sub(r'^\d+\.', '', hotel)) for hotel in hotel_names_with_numbers]

    review_counts = []
    for count_review in soup.find_all("span", class_="biGQs _P pZUbB hmDzD"):
        span_tag = count_review.find("span", class_="S4")
        if span_tag:
            review_counts.append(span_tag.text)

    review_counts_int = [int(re.sub(r'\.', '', re.search(r'\d+(\.\d+)*', review).group())) for review in review_counts]
    

    return pd.DataFrame({"name":cleaned_hotel_names, "count":review_counts_int })


def get_button_hrefs(soup):
    hrefs = []
    buttons = soup.find_all('button', class_='ypcsE _S wSSLS')
    
    for button in buttons:
        a_tag = button.find('a', href=True)
        if a_tag:
            hrefs.append(a_tag['href'])
    
    return hrefs

def get_dynamic_links(hotels):
    hotel_review_links = []
    root = "https://www.tripadvisor.com.tr"
    for hotel in hotels:
        part = hotel
        full= root+part
        # print(f"\nLinkler: {full}")
        
        location = full.find('Reviews')
        
        # print(f"Linkteki Reviewsin Konumu: {location}")
        full = full[:(location+7)] +"-or{}"+ full[(location+7):]
        # print(f"Dinamik Link: {full}")
        
        hotel_review_links.append(full)
    
    return hotel_review_links


def get_review_by_url(url):
    
    try:
        review_page = parse_html(fetch_html(url))
        reviews = review_page.find_all("div", class_="_T FKffI bmUTE")
        
        texts=[]
        for review in reviews:
            span_element = review.find("span", class_="orRIx Ci _a C")
            if span_element is not None:
                texts.append(span_element.text)
        
        return texts
    except :
        return " "

In [72]:
url = "https://www.tripadvisor.com.tr/Hotels-g297962-Antalya_Turkish_Mediterranean_Coast-Hotels.html"
response = fetch_html(url=url)

HTTP hatası: 403 Client Error: Forbidden for url: https://www.tripadvisor.com.tr/Hotels-g297962-Antalya_Turkish_Mediterranean_Coast-Hotels.html


In [56]:
soup = parse_html(response)

In [57]:
hotels = get_button_hrefs(soup)

In [58]:
hotels

['/Hotel_Review-g20116893-d572784-Reviews-Concorde_De_Luxe_Resort-Kemeragzi_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 '/Hotel_Review-g297962-d1804746-Reviews-Megasaray_Westbeach_Antalya-Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 '/Hotel_Review-g15300585-d557049-Reviews-Lara_Barut_Collection-Muratpasa_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 '/Hotel_Review-g20116893-d568401-Reviews-Liberty_Hotels_Lara-Kemeragzi_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 '/Hotel_Review-g17951017-d307373-Reviews-Swandor_Hotels_Resorts_Topkapi_Palace-Aksu_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 '/Hotel_Review-g20116893-d5982512-Reviews-Aska_Lara_Resort_Spa-Kemeragzi_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 '/Hotel_Review-g20116893-d599140-Reviews-Limak_Lara_Deluxe_Hotel_Resort-Kemeragzi_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 '/Hotel_Review-g17951017-d15268740-Reviews-Titanic_Mardan_Palace-Aksu_Antalya_Turkish_Mediterranean_Coast.htm

In [59]:
hotel_review_links = get_dynamic_links(hotels)

In [60]:
hotel_review_links

['https://www.tripadvisor.com.tr/Hotel_Review-g20116893-d572784-Reviews-or{}-Concorde_De_Luxe_Resort-Kemeragzi_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g297962-d1804746-Reviews-or{}-Megasaray_Westbeach_Antalya-Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g15300585-d557049-Reviews-or{}-Lara_Barut_Collection-Muratpasa_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g20116893-d568401-Reviews-or{}-Liberty_Hotels_Lara-Kemeragzi_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g17951017-d307373-Reviews-or{}-Swandor_Hotels_Resorts_Topkapi_Palace-Aksu_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 'https://www.tripadvisor.com.tr/Hotel_Review-g20116893-d5982512-Reviews-or{}-Aska_Lara_Resort_Spa-Kemeragzi_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS',
 'https://www.tripadvisor.com.tr/H

In [62]:
get_review_by_url(hotel_review_links[0].format(5))

HTTP hatası: 403 Client Error: Forbidden for url: https://www.tripadvisor.com.tr/Hotel_Review-g20116893-d572784-Reviews-or5-Concorde_De_Luxe_Resort-Kemeragzi_Antalya_Turkish_Mediterranean_Coast.html#REVIEWS


' '

In [63]:
hotel_df = get_namesandcount(soup)
hotel_df["review_link"] = hotel_review_links

In [64]:
results = []
for idx, row in hotel_df.iterrows():
    links = []
    for rng in range(0, row["count"], 5):
        if rng ==0: continue
        link = row["review_link"].format(rng)
        links.append(link)
    results.append(links)

hotel_df["dynamic_links"] = results

In [68]:
hotel_df

,name,count,review_link,dynamic_links
0,Concorde De Luxe Resort,6641,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
1,Megasaray WestBeach Antalya,1379,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
2,Lara Barut Collection,10208,https://www.tripadvisor.com.tr/Hotel_Review-g1...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
3,Liberty Hotels Lara,7356,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
4,Swandor Hotels Resorts Topkapi Palace,4256,https://www.tripadvisor.com.tr/Hotel_Review-g1...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
5,Aska Lara Resort Spa,7635,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
6,Limak Lara Deluxe Hotel Resort,6610,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
7,Titanic Mardan Palace,4149,https://www.tripadvisor.com.tr/Hotel_Review-g1...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
8,Rixos Downtown Antalya,3518,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...
9,Grand Park Lara,3235,https://www.tripadvisor.com.tr/Hotel_Review-g2...,[https://www.tripadvisor.com.tr/Hotel_Review-g...


In [77]:
hotel_df["count"].sum()/5/2

9318.1

In [26]:
# hotel_df.to_csv("hotels_with_links.csv", index=False)

# Scrape reviews

In [27]:
hotel_df = pd.read_csv("hotels_with_links.csv")

In [28]:
text_list = []
for idx, row in hotel_df.iterrows():
    text= []
    for i,url in enumerate(row["dynamic_links"]):
        print(f"Hotel Name: {row['name']} Count: {(i+1)*5}")
        print(url)
        print("----------------------------------------------")
        txt = get_review_by_url(url)
        time.sleep(5)
        text.extend(txt)
    
    print(f"New hotel\n\n")
    text_list.append(text)

Hotel Name: Concorde De Luxe Resort Count: 5
[
----------------------------------------------
Hotel Name: Concorde De Luxe Resort Count: 10
'
----------------------------------------------
Hotel Name: Concorde De Luxe Resort Count: 15
h
----------------------------------------------
Hotel Name: Concorde De Luxe Resort Count: 20
t
----------------------------------------------
Hotel Name: Concorde De Luxe Resort Count: 25
t
----------------------------------------------
Hotel Name: Concorde De Luxe Resort Count: 30
p
----------------------------------------------


KeyboardInterrupt: 

In [ ]:
text

['Concorde Otel, mükemmel hizmet kalitesi ve konforuyla öne çıkıyor. Şık ve modern odaları, lezzetli yemekler sunan restoranları ve güler yüzlü personeli ile misafirlerine unutulmaz bir deneyim yaşatıyor. Merkezi konumu sayesinde şehirdeki önemli noktalara kolayca ulaşım imkanı sunuyor.',
 'Buraya geldiğim için çok mutluyum buradaki her şeyi genel anlamda çok beğendim yemekleri çok lezzetli sahili plajı denizi havuzu herşeyiyle mükemmel çalışanlar güler yüzlü ve ilgili yemekler çok lezzetliydi özellikle sahildeki Boyan partiye bayıldım animasyon dans yolları gece etkinlikleri canlı müziği herşeyiyle kesinlikle gelmeye değer kesinlikle ailemle tekrar geleceğim',
 'Herkes cok kibar ve iyi.. Temizlik çok güzel..Restaurant harika yemekler çok lezzetli.. Burada kötü bir ṣey hiç görmedim..Denizi havuzu aqua parkì harika.. Harika dans shovları var.. Aktiviteler harika.. Voleybol, dart,yoga, step aerobik gibi harika aktiviteler var.. Canlı müzik ve dj performans gerçekten harika.. Çalisanlar ç